In [8]:
# Smolagent huggingface_hub

In [9]:

import os
from typing import List, Dict, Any
from smolagents import CodeAgent, DuckDuckGoSearchTool, tool

In [10]:
# 1) Set your OpenAI key (or have it in your env already)
from openai import OpenAI

client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),  # or set OPENAI_API_KEY in your env
)


In [11]:
import os
from typing import List, Dict, Any, Optional
from openai import OpenAI
from smolagents.models import ChatMessage

class OpenAIChatModel:
    def __init__(
        self,
        model_name: str = "gpt-4",
        temperature: float = 0.7,
        max_tokens: int = 512,
        api_key: Optional[str] = None,
    ):
        self.client = OpenAI(api_key=api_key or os.getenv("OPENAI_API_KEY"))
        self.model_name = model_name
        self.temperature = temperature
        self.max_tokens = max_tokens

    def __call__(
        self,
        messages: List[Dict[str, Any]],
        stop_sequences: Optional[List[str]] = None,
        **kwargs,
    ) -> ChatMessage:
        resp = self.client.chat.completions.create(
            model=self.model_name,
            messages=messages,
            temperature=self.temperature,
            max_tokens=self.max_tokens,
            stop=stop_sequences,
            **kwargs,
        )
        choice = resp.choices[0].message
        content = choice.content.strip()
        # wrap into ChatMessage so CodeAgent can do `.content` on it
        return ChatMessage(
            role=choice.role,
            content=content,
            raw=resp
        )


In [12]:
# 3) (Optional) a simple custom tool for demonstration
@tool
def echo_tool(text: str) -> str:
    """
    Echo back whatever you send in.

    Args:
        text (str): The input text to echo back.

    Returns:
        str: A string that echoes back the input text.
    """
    return f"You said: {text}"

In [13]:
# 4) Build your agent with OpenAIChatModel instead of HfApiModel
agent = CodeAgent(
    tools=[
        DuckDuckGoSearchTool(),
        echo_tool,               # your own tools continue to work
    ],
    model=OpenAIChatModel(),     # ← use your new wrapper here
)

In [15]:

# 5) Run a query
result = agent.run("what is happening today in Rome.")
print(result)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ what is happening today in Rome.                                                                                │
│                                                                                                                 │
╰─ OpenAIChatModel -  ────────────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  rome_news_today = web_search(query="Rome news today")                                                            
  print(rome_news_today)                                                                                           
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
## Search Results

[News in Italy - Rome's local English new - Wanted in Rome](https://www.wantedinrome.com/news)
News in Italy and Rome's local English news. Since 1985 we've been providing news on culture, politics, sports and 
all major happenings in Rome. ... Rome marks Natale di Roma on 21 April.Rome celebrates 2,778 years today, Monday 
21 April, in a popular birthday tradition called Natale di Roma.The annual celebration - known in ancient times as 
...

[Pope Francis' funeral live updates: Pontiff laid to rest at Rome's St 
...](https://www.nbcnews.com/world/the-vatican/live-blog/pope-francis-funeral-live-updates-vatican-holy-catholic-ch
urch-death-rcna202949)
Watch live coverage of Pope Francis' funeral ceremony at the Vatican on NBC News. Pope Francis died on Easter 
Monday at age 88. ... to Rome last week for the Jubilee for Teenagers, an event that ...

[Rome - latest news, breaking stories and comment - The Independent](https://www.independent.co.uk/topic/rome)
A rare peek at the hidden waterworks behind Rome's Trevi Fountain News Italy's Constitutional Court allows singles 
to adopt foreign minors, ending a 40-year ban

[Wanted in Rome | Italy's news in English](https://www.wantedinrome.com/)
Check out the latest English news in Rome and Italy. News and events from Italy. News about politics, business, 
sports, health, and fashion. 16.5 C. Rome (IT) Thu, 24 April 2025. Italy's news in English ... The City celebrates 
2,778 years today. History 21 Apr, 2025. Vance meets Pope Francis briefly on Easter Sunday. Politics 21 Apr, 2025. 
Rome ...

[Pope Francis funeral live: Pontiff to be laid to rest after 
...](https://www.independent.co.uk/news/world/europe/pope-francis-funeral-vatican-rome-trump-live-latest-news-b2739
997.html)
Pope Francis funeral: Watch live updates today as world leaders arrive at Rome . ... (Sky News) 26 April 2025 
11:48. Mapped: The Pope's route to his final resting place.

[Pope Francis funeral live updates: Pontiff reaches final ... - USA 
TODAY](https://www.usatoday.com/story/news/world/2025/04/26/pope-francis-funeral-live-updates/83247476007/)
Gussie joined Vatican News in October 2024 after more than two years at Rome Reports TV News Agency, a news agency 
focused on covering the pope and the Vatican, according to her LinkedIn page.

[Rome, IT Breaking News Headlines Today | Ground News](https://ground.news/interest/rome)
Stay current with all the latest and breaking news about Rome, compare headlines and perspectives between news 
sources on stories happening today.In total, 5,172 stories have been published about Rome which Ground News has 
aggregated in the past 3 months.

[St Peter's Square closes ahead of Pope Francis' 
funeral](https://www.euronews.com/2025/04/25/world-leaders-converge-in-rome-to-honour-pope-francis-live-updates)
Meanwhile, over 100 delegations are on their way to Rome to pay respects to the late leader of the Catholic Church,
who will be laid to rest in the Italian capital on Saturday. Follow the live ...

[Pope Francis latest: Security ramping up ahead of Pope's funeral - Sky 
News](https://news.sky.com/story/pope-francis-latest-queue-to-see-pope-francis-lying-in-state-closes-as-world-leade
rs-descend-on-rome-for-funeral-12846482)
The US president has now touched down in Rome in the last few minutes. 21:49:01 Death of football-loving Pope comes
as president of club he supported faces corruption claims

[Tens of thousands join pro-Europe rally in Rome, amid worries over 
...](https://apnews.com/article/italy-europe-rally-rome-rearm-defense-e14c1e3df35d9c328bb23644bd3adb75)
ROME (AP) — Tens of thousands of Italians joined a pro-Europe rally in Rome's city center Saturday, waving blue 
European Union flags in a sign of support and unity as a European push for rearmament divides the country.. The 
initiative, supported by most of the center-left opposition parties, despite their different positions, was 
launched by Italian journalist Michele Serra at 

[Step 1: Duration 10.01 seconds]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Error in generating model output:
Error code: 400 - {'error': {'message': "Invalid value: 'tool-call'. Supported values are: 'system', 'assistant', 
'user', 'function', 'tool', and 'developer'.", 'type': 'invalid_request_error', 'param': 'messages[3\].role', 
'code': 'invalid_value'}}

[Step 2: Duration 3.25 seconds]

AgentGenerationError: Error in generating model output:
Error code: 400 - {'error': {'message': "Invalid value: 'tool-call'. Supported values are: 'system', 'assistant', 'user', 'function', 'tool', and 'developer'.", 'type': 'invalid_request_error', 'param': 'messages[3].role', 'code': 'invalid_value'}}